# Imports

In [ ]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
from tqdm import tqdm

import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import string
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
import nltk

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from gensim.models.nmf import Nmf
from collections import Counter
from operator import itemgetter
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

In [ ]:
from top2vec import Top2Vec
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

# Data Pre-processing

In [ ]:
df = pd.read_csv('/home/703124579/DG_TopicModelling/Notes Report Aug2021-ticket_title-ERC_NEXT_GEN_POS_tech4.csv')

# df = pd.read_csv("/home/703124579/DG_TopicModelling/Notes Report Aug2021-issue-ERC_POS_tech4.csv")
# df = pd.read_csv("/home/703272232/Projects/DollarGeneral/Clustering/sep_oct-troubleshooting-ERC_POS_NEX_GEN.csv")
# df = df.dropna(subset=['Ticket_Title'])

In [ ]:
df = df.drop(labels=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis=1)

In [ ]:
# df = df[:12000]

In [ ]:
df.shape

In [ ]:
df.columns

# Topic2Vec

## Pre-Processing

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# stop_words = set(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))

def to_string(item):
    item_str = str(item)
    item_str = ' '.join([word for word in item_str.split() if word not in STOPWORDS])
    return item_str
 

## Get-Topics

In [ ]:
def top2vec_tr_tops(df, sub_cat=''):
    tr = df.dropna(subset=['Troubleshooting'])
    tr["Troubleshooting"] = tr["Troubleshooting"].progress_apply(to_string)
    documents_tr = tr["Troubleshooting"].to_list()
    print('modeling')
    model_tr = Top2Vec(documents_tr, speed="learn", workers=8)
    model_tr.save("top2vec-troubleshooting" if sub_cat=='' else "top2vec-troubleshooting-"+str(sub_cat))
    tr["top2vec_topic"] = model_tr.doc_top
    print('saving to df')
    tr["top2vec_topic_words"] = ''
    tr["top2vec_topic_word_scores"] = ''
    for i,topic in tqdm(enumerate(model_tr.doc_top)):
        tr["top2vec_topic_words"].iloc[i] = list(model_tr.topic_words[topic])
        tr["top2vec_topic_word_scores"].iloc[i] = list(model_tr.topic_word_scores[topic])

    return tr

# NMF

## Pre-Processing

In [ ]:
# Contraction map
c_dict = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "I would",
    "i'd've": "I would have",
    "i'll": "I will",
    "i'll've": "I will have",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'alls": "you alls",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you you will",
    "you'll've": "you you will have",
    "you're": "you are",
    "you've": "you have"
}

# Compiling the contraction dict
c_re = re.compile('(%s)' % '|'.join(c_dict.keys()))

# List of stop words
add_stop = ['said', 'say', '...', 'like', 'cnn', 'ad', 'troubleshoot']
stop_words = ENGLISH_STOP_WORDS.union(add_stop)

# List of punctuation
punc = list(set(string.punctuation))


# Splits words on white spaces (leaves contractions intact) and splits out
# trailing punctuation
def casual_tokenizer(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens


def expandContractions(text, c_re=c_re):
    def replace(match):
        return c_dict[match.group(0)]
    return c_re.sub(replace, text)


def process_text(text):
    text = casual_tokenizer(text)
    text = [each.lower() for each in text]
    text = [re.sub('[0-9]+', '', each) for each in text]
    text = [expandContractions(each, c_re=c_re) for each in text]
    text = [SnowballStemmer('english').stem(each) for each in text]
    text = [w for w in text if w not in punc]
    text = [w for w in text if w not in stop_words]
    text = [each for each in text if len(each) > 1]
    text = [each for each in text if ' ' not in each]
    return text


def top_words(topic, n_top_words):
    return topic.argsort()[:-n_top_words - 1:-1]  


def topic_table(model, feature_names, n_top_words):
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        t = (topic_idx)
        topics[t] = [feature_names[i] for i in top_words(topic, n_top_words)]
    return pd.DataFrame(topics)


def whitespace_tokenizer(text): 
    pattern = r"(?u)\b\w\w+\b" 
    tokenizer_regex = RegexpTokenizer(pattern)
    tokens = tokenizer_regex.tokenize(text)
    return tokens


# Funtion to remove duplicate words
def unique_words(text): 
    ulist = []
    [ulist.append(x) for x in text if x not in ulist]
    return ulist


def word_count(text):
    return len(str(text).split(' '))

In [ ]:
from tqdm import tqdm
tqdm.pandas()

## Get-Topics

In [ ]:
def nmf_tr_tops(df, sub_cat=''):
    df_tt = df.dropna(subset=['Troubleshooting'])
    df_tt['processed_text'] = df_tt['Troubleshooting'].progress_apply(process_text)
    texts = df_tt['processed_text']
    dictionary = Dictionary(texts)

    # Filter out extremes to limit the number of features
    dictionary.filter_extremes(
        no_below=3,
        no_above=0.85,
        keep_n=5000
    )

    # Create the bag-of-words format (list of (token_id, token_count))
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Create a list of the topic numbers we want to try
    topic_nums = list(np.arange(5, 70 + 1, 3))

    # Run the nmf model and calculate the coherence score
    # for each number of topics
    coherence_scores = []

    for num in topic_nums:
        nmf = Nmf(
            corpus=corpus,
            num_topics=num,
            id2word=dictionary,
            chunksize=2000,
            passes=5,
            kappa=.1,
            minimum_probability=0.01,
            w_max_iter=300,
            w_stop_condition=0.0001,
            h_max_iter=100,
            h_stop_condition=0.001,
            eval_every=10,
            normalize=True,
            random_state=42
        )
    
    # Run the coherence model to get the score
    cm = CoherenceModel(
        model=nmf,
        texts=texts,
        dictionary=dictionary,
        coherence='c_v'
    )
    
    coherence_scores.append(round(cm.get_coherence(), 5))

    # Get the number of topics with the highest coherence score
    scores = list(zip(topic_nums, coherence_scores))
    best_num_topics = sorted(scores, key=itemgetter(1), reverse=True)[0][0]

    # Creating model
    tfidf_vectorizer = TfidfVectorizer(
        min_df=3,
        max_df=0.85,
        max_features=5000,
        ngram_range=(1, 2),
        preprocessor=' '.join
    )

    tfidf = tfidf_vectorizer.fit_transform(texts)

    # Save the feature names for later to create topic summaries
    tfidf_fn = tfidf_vectorizer.get_feature_names()

    # Run the nmf model
    nmf = NMF(
        n_components=best_num_topics,
        init='nndsvd',
        max_iter=500,
        l1_ratio=0.0,
        solver='cd',
        alpha=0.0,
        tol=1e-4,
        random_state=42
    ).fit(tfidf)
    
    docweights = nmf.transform(tfidf_vectorizer.transform(texts))
    n_top_words = 8

    topic_df = topic_table(
        nmf,
        tfidf_fn,
        n_top_words
    ).T

    # Cleaning up the top words to create topic summaries
    topic_df['topics'] = topic_df.apply(lambda x: [' '.join(x)], axis=1) # Joining each word into a list
    topic_df['topics'] = topic_df['topics'].str[0]  # Removing the list brackets
    topic_df['topics'] = topic_df['topics'].apply(lambda x: whitespace_tokenizer(x)) # tokenize
    topic_df['topics'] = topic_df['topics'].apply(lambda x: unique_words(x))  # Removing duplicate words
    topic_df['topics'] = topic_df['topics'].apply(lambda x: [' '.join(x)])  # Joining each word into a list
    topic_df['topics'] = topic_df['topics'].str[0]  # Removing the list brackets

    df_tt["nmf_topics"] = list(docweights.argmax(axis=1))
    df_tt["nmf_topic_words"] = ''

    for i,topic in tqdm(enumerate(list(docweights.argmax(axis=1)))):
        df_tt["nmf_topic_words"].iloc[i] = topic_df["topics"].iloc[topic]
    
    df_tt = df_tt.drop('processed_text', axis=1)
    return df_tt

# BERT-Topic

## Pre-Processing

In [ ]:
from bertopic import BERTopic

In [ ]:
from string import digits
from tqdm import tqdm
import unicodedata
tqdm.pandas()
remove_digits = str.maketrans('', '', digits)

def clean_text(text):
    res = str(text).translate(remove_digits)
    return res

def to_utf8(text):
    # Convert to ascii characters and return text in utf8 format
    return unicodedata.normalize('NFKD', text).encode("ascii", "ignore").decode('utf-8')


def getTopicKeywords(model, topic):
    predictions = model.get_topic(topic)
    keywords = []
#     print(predictions)
    print(type(predictions),end='')
    for pre in predictions:
        keywords.append(pre[0])
    return keywords

def bert_pre_pro(_df):
    _df = _df.dropna(subset=['Troubleshooting'])
    _df["Troubleshooting"] = _df["Troubleshooting"].progress_apply(clean_text)
    _df["Troubleshooting"] = _df["Troubleshooting"].progress_apply(to_utf8)
    return _df

## Utils

In [ ]:
def upsample_records(_df):
    _df = _df.dropna(subset=['Troubleshooting'])
    while(_df.shape[0]<400):
        _df = _df.append(_df)
    return _df

In [ ]:
def drop_duplicates(_df):
    return _df.drop_duplicates(subset = ['Ticket_Title', 'Ticket_Number', 'Notes'])

## Get Topics

In [ ]:
def bert_tr_tops(df, _nr_topics='auto', sub_cat=''):
    text = df["Troubleshooting"].tolist()
    model = BERTopic(calculate_probabilities=True, nr_topics=_nr_topics, top_n_words=20)
    print("modeling")
    topics, probabilities = model.fit_transform(text)

    words =[]
    tops =[]
    for t in topics:
        keywords = getTopicKeywords(model, t)
        words.append(keywords)
        tops.append(t)

    df['bert_topic'] = tops
    df['bert_topic_words'] = words

    return df

# Main

## --

In [ ]:
topics = dict(df["Technique4_Topic_No"].value_counts())

In [ ]:
print(topics)

In [ ]:
# df["top2vec_topic"] = ''
# df["top2vec_topic_words"] = ''
# df["top2vec_topic_word_scores"] = ''
# df["nmf_topics"] = ''
# df["nmf_topic_words"] = ''
df["bert_topic"] = ''
df["bert_topic_words"] = ''

## top2vec

In [ ]:
for top_i in tqdm(topics.keys()):
    try:
        tr = top2vec_tr_tops(df.loc[df["Technique4_Topic_No"]==top_i], sub_cat=top_i)

        top2vec_topic = dict(tr["top2vec_topic"])
        top2vec_topic_words = dict(tr["top2vec_topic_words"])
        top2vec_topic_word_scores = dict(tr["top2vec_topic_word_scores"])

        for i in tqdm(top2vec_topic.keys()):
            top2vec_topic_loc = df.columns.get_loc('top2vec_topic')
            top2vec_topic_words_loc = df.columns.get_loc('top2vec_topic_words')
            top2vec_topic_word_scores_loc = df.columns.get_loc('top2vec_topic_word_scores')
            df.iloc[i, top2vec_topic_loc] = top2vec_topic[i]
            df.iloc[i, top2vec_topic_words_loc] = str(top2vec_topic_words[i])
            df.iloc[i, top2vec_topic_word_scores_loc] = str(top2vec_topic_word_scores[i])
    except:
        print("Skipping top_i:", top_i)
        continue

## nmf

In [ ]:
for top_i in tqdm(topics.keys()):
    tr = nmf_tr_tops(df.loc[df["Technique4_Topic_No"]==top_i], sub_cat=top_i)

    nmf_topics = dict(tr["nmf_topics"])
    nmf_topic_words = dict(tr["nmf_topic_words"])

    for i in tqdm(nmf_topics.keys()):
        nmf_topics_loc = df.columns.get_loc('nmf_topics')
        nmf_topic_words_loc = df.columns.get_loc('nmf_topic_words')
        df.iloc[i, nmf_topics_loc] = nmf_topics[i]
        df.iloc[i, nmf_topic_words_loc] = str(nmf_topic_words[i])

## bert

In [ ]:
df = bert_pre_pro(df)

In [ ]:
skipped_topics = []

for top_i in tqdm(topics.keys()):
    try:
        tr = bert_tr_tops(df.loc[df["Technique4_Topic_No"]==top_i], sub_cat=top_i)
        bert_topic = dict(tr["bert_topic"])
        bert_topic_words = dict(tr["bert_topic_words"])

        for i in tqdm(bert_topic.keys()):
            df.loc[[i], 'bert_topic'] = bert_topic[i]
            df.loc[[i], 'bert_topic_words'] = str(bert_topic_words[i])
    except Exception as e:
        print(e)
        skipped_topics.append(top_i)
        print("Skipping top_i:", top_i)
        pass

In [ ]:
print(skipped_topics)

In [ ]:
failed_topics = []

for top_i in tqdm(skipped_topics):
    try:
        print(top_i)
        # upsampling records (>400)
        tr = upsample_records(df.loc[df["Technique4_Topic_No"]==top_i])
        tr = bert_pre_pro(tr)
        tr = bert_tr_tops(tr.loc[tr["Technique4_Topic_No"]==top_i], sub_cat=top_i)
        # removing duplicate records created while upsampling
        tr = drop_duplicates(tr)
        
        bert_topic = dict(tr["bert_topic"])
        bert_topic_words = dict(tr["bert_topic_words"])

        for i in tqdm(bert_topic.keys()):
            df.loc[[i], 'bert_topic'] = bert_topic[i]
            df.loc[[i], 'bert_topic_words'] = str(bert_topic_words[i])
    except Exception as e:
        print(e)
        failed_topics.append(top_i)
        print("Failed Topics top_i:", top_i)
        pass

In [ ]:
print(failed_topics)

### Merge single record buckets

In [ ]:
def get_bucket_size(_df, _column_name='bert_topic'):
    bucket_size = []
    for top_i,count in dict(_df[_column_name].value_counts()).items():
        if count<=1:
            bucket_size.append((int(top_i), count))
    bucket_size.sort()
    return bucket_size
b = []
for top_i in tqdm(topics.keys()):
    buckets = get_bucket_size(df.loc[df["Technique4_Topic_No"]==top_i])
    print(buckets); b.append(buckets);
    df = df.replace({"bert_topic":buckets}, -2)

# Troubleshooting

In [ ]:
df.loc[df["Technique4_Topic_No"]==-1]#.dropna(subset=['Troubleshooting'])

# Save Results

In [ ]:
df.to_csv("Notes Report Aug2021-troubleshooting-ERC_POS_tech4.csv")

# Rough

In [ ]:
# import collections


# for top_i in range(-1,30,1):
#     print('\ntop_i:',top_i)
#     try:
#         t2v = dict(df.loc[df["Technique4_Topic_No"]==top_i].dropna(subset=['Troubleshooting'])["top2vec_topic"].value_counts())
#         t2v = collections.OrderedDict(sorted(t2v.items()))
#         print('top2vec', t2v)
#     except:
#         pass
#     try:
#         nmf = dict(df.loc[df["Technique4_Topic_No"]==top_i].dropna(subset=['Troubleshooting'])["nmf_topics"].value_counts())
#         nmf = collections.OrderedDict(sorted(nmf.items()))
#         print('nmf', nmf)
#     except:
#         pass
#     try:
#         b = dict(df.loc[df["Technique4_Topic_No"]==top_i].dropna(subset=['Troubleshooting'])["bert_topic"].value_counts())
#         b = collections.OrderedDict(sorted(b.items()))
#         print('bert', b)
#     except:
#         pass

NMF
Top2Vec
BERT UMAP
BERT-Topic